In [156]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andreshat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [224]:
train = pd.read_csv('../data/train.csv', index_col = 'id')
test = pd.read_csv('../data/test.csv', index_col = 'id')

In [225]:
stop_words = set(stopwords.words('english'))

def process(text):
    res = re.sub('<.*?>', ' ', text)
    res = re.sub('\W', ' ', res)
    res = re.sub('\s+[a-zA-Z]\s+', ' ', res)
    res = re.sub('\s+', ' ', res)
    res = re.sub(r'\d+', '', res)
    word_list = word_tokenize(res)
#     word_list = [WordNetLemmatizer().lemmatize(w) for w in word_list]
#     word_list = [i for i in word_list if len(i) >= 3]
    filtered_res = " ".join([w for w in word_list if w not in stop_words])
    return filtered_res

In [226]:
train['processed'] = train['review'].apply(lambda x: process(x))
test['processed'] = test['review'].apply(lambda x: process(x))

In [227]:
train['processed'][2]

'It Libby talking Desmond flashback anyone confused past like end hospital Hurley know despited Libby dying season character explored season get answers questions surrounding BTW great episode It really great cliffhanger interesting questions like happened Eko Lock four toe statue wait till season Lost rules hope unanswered questions answered loved explained plane actually crashed Desmond manage type numbers time'

In [252]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.99, ngram_range = (1, 3))
x_train_vector = tfidf.fit_transform(train['processed'])
y_train = train['sentiment'].values

In [261]:
params = {'C': [0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94]}
#           'multi_class': ['ovr', 'crammer_singer'],
#           'loss': ['hinge', 'squared_hinge']}
gs = GridSearchCV(LinearSVC(), params, scoring = 'f1', n_jobs = -1)
gs.fit(x_train_vector, y_train)

GridSearchCV(estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': [0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93,
                               0.94]},
             scoring='f1')

In [262]:
x_test_vector = tfidf.transform(test['processed'])
test_pred = gs.best_estimator_.predict(x_test_vector)

sub = pd.read_csv('../data/submission.csv')
sub['sentiment'] = test_pred
# sub.to_csv('./my_sub.csv')

In [263]:
gs.best_score_

0.9097535008469129

In [264]:
gs.best_params_

{'C': 0.91}

In [265]:
sub.to_csv("submission_linear_svm_gs.csv", index=False)